In [1]:
import math
import copy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
# -----------------------------
# Dataset placeholder - adapt to your files
# -----------------------------
class MyTimeSeriesDataset(Dataset):
    """
    Expects:
      X: np.array (N, seq_len, n_features)
      y: np.array (N,)  # scalar DO target value
    Replace the `from_arrays` usage with file loading/parsing.
    """
    def __init__(self, X: np.ndarray, y: np.ndarray):
        assert X.ndim == 3
        assert len(X) == len(y)
        self.X = X.astype(np.float32)
        self.y = y.astype(np.float32)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

    @staticmethod
    def from_arrays(X, y):
        return MyTimeSeriesDataset(np.array(X), np.array(y))

In [3]:
# -----------------------------
# Utility metrics (paper metrics)
# -----------------------------
def rmse(y_true, y_pred):
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))

def mape(y_true, y_pred):
    # avoid division by zero
    denom = np.where(np.abs(y_true) < 1e-8, 1e-8, y_true)
    return float(np.mean(np.abs((y_true - y_pred) / denom)) * 100.0)

def r2_score_np(y_true, y_pred):
    ss_res = np.sum((y_true - y_pred)**2)
    ss_tot = np.sum((y_true - np.mean(y_true))**2)
    return float(1.0 - ss_res / ss_tot) if ss_tot != 0 else 0.0

def index_of_agreement(y_true, y_pred):
    num = np.sum((y_true - y_pred)**2)
    denom = np.sum((np.abs(y_true - np.mean(y_true)) + np.abs(y_pred - np.mean(y_true)))**2)
    return float(1 - num / denom) if denom != 0 else 0.0

def nash_sutcliffe(y_true, y_pred):
    num = np.sum((y_pred - y_true)**2)
    denom = np.sum((y_true - np.mean(y_true))**2)
    return float(1 - num / denom) if denom != 0 else 0.0

In [4]:
# -----------------------------
# Model blocks
# -----------------------------
class ResBlock1D(nn.Module):
    def __init__(self, in_ch, out_ch, kernel_sizes, stride=1):
        super().__init__()
        # kernel_sizes: list/tuple e.g. [1,3,1]
        ks1, ks2, ks3 = kernel_sizes
        # Use padding to keep control of length
        self.conv1 = nn.Conv1d(in_ch, out_ch, kernel_size=ks1, stride=stride, padding=ks1//2)
        self.conv2 = nn.Conv1d(out_ch, out_ch, kernel_size=ks2, stride=1, padding=ks2//2)
        self.conv3 = nn.Conv1d(out_ch, out_ch, kernel_size=ks3, stride=1, padding=ks3//2)
        self.relu = nn.ReLU()
        self.ln = nn.LayerNorm(out_ch)

        # if in_ch != out_ch or stride !=1, we make a projection
        if in_ch != out_ch or stride != 1:
            self.shortcut = nn.Conv1d(in_ch, out_ch, kernel_size=1, stride=stride)
        else:
            self.shortcut = None

    def forward(self, x):  # x: [batch, channels, seq_len]
        out = self.conv1(x)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.relu(out)
        out = self.conv3(out)  # [batch, out_ch, seq_len']
        # layernorm across channel dim: LayerNorm expects last dim features -> transpose
        out_t = out.transpose(1, 2)  # [batch, seq_len, channels]
        out_t = self.ln(out_t)
        out = out_t.transpose(1, 2)
        if self.shortcut is not None:
            sc = self.shortcut(x)
        else:
            sc = x
        return self.relu(out + sc)

In [5]:
# -----------------------------
# Fusion model: ResNet branch, BiLSTM branch, Attention branch, concat -> FC
# -----------------------------
class DOTransferModel(nn.Module):
    def __init__(self, n_features, seq_len, device='cpu'):
        super().__init__()
        self.device = device
        # We'll implement three ResNet blocks as parallel branch: but paper fed same input to 3 branches
        # Branch 1: ResNet Block 1 -> out filters 32
        self.res1 = ResBlock1D(in_ch=n_features, out_ch=32, kernel_sizes=(1,3,1), stride=1)
        self.res2 = ResBlock1D(in_ch=32, out_ch=64, kernel_sizes=(1,5,1), stride=2)  # stride 2 as paper
        self.res3 = ResBlock1D(in_ch=64, out_ch=128, kernel_sizes=(1,3,1), stride=1)

        # After conv, do temporal global average pooling -> vector
        # BiLSTM branch: three stacked BiLSTM layers configured as in paper: units 16,32,64
        self.bilstm1 = nn.LSTM(input_size=n_features, hidden_size=16, num_layers=1, batch_first=True, bidirectional=True)
        self.bilstm2 = nn.LSTM(input_size=32, hidden_size=32, num_layers=1, batch_first=True, bidirectional=True)
        self.bilstm3 = nn.LSTM(input_size=64, hidden_size=64, num_layers=1, batch_first=True, bidirectional=True)
        # We'll pass input sequentially through lstm1->lstm2->lstm3 by projecting features

        # Attention branch: we'll use three multihead attention modules (different embed dims & heads)
        # MultiheadAttention in PyTorch expects (seq_len, batch, embed_dim) if batch_first=False
        self.attn_proj1 = nn.Linear(n_features, 16)
        self.mha1 = nn.MultiheadAttention(embed_dim=16, num_heads=4, batch_first=True)
        self.attn_proj2 = nn.Linear(n_features, 32)
        self.mha2 = nn.MultiheadAttention(embed_dim=32, num_heads=4, batch_first=True)
        self.attn_proj3 = nn.Linear(n_features, 64)
        self.mha3 = nn.MultiheadAttention(embed_dim=64, num_heads=8, batch_first=True)

        # Combine sizes:
        # ResNet final filters = 128, we'll global pool over time -> 128
        # BiLSTM final hidden (bidirectional) -> 64*2 = 128 (we'll take last hidden)
        # Attention branches -> produce pooled feature sizes: 16 + 32 + 64 = 112
        combined = 128 + 128 + (16 + 32 + 64)
        # Fully connected projection head from paper: two FC layers 128, 128 then 1
        self.fc1 = nn.Linear(combined, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc_out = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):  # x: [batch, seq_len, n_features]
        b, seq_len, nfeat = x.shape

        # ---- ResNet branch ----
        # conv1d expects (batch, channels, seq_len)
        xr = x.transpose(1, 2)  # [b, n_features, seq_len]
        xr = self.res1(xr)
        xr = self.res2(xr)
        xr = self.res3(xr)  # [b, 128, seq_len']
        # global average pooling across time dimension
        xr_pool = xr.mean(dim=2)  # [b, 128]

        # ---- BiLSTM branch ----
        xb = x  # [b, seq_len, n_features]
        out1, _ = self.bilstm1(xb)    # out1: [b, seq_len, 32]
        out2, _ = self.bilstm2(out1)  # out2: [b, seq_len, 64]
        out3, (hn, cn) = self.bilstm3(out2)  # out3: [b, seq_len, 128]
        # hn has shape (num_layers*2, b, hidden_size) => take last layer both directions
        # We'll flatten last layer states
        # hn[-2] is last forward, hn[-1] is last backward (since num_layers=1)
        if hn.shape[0] >= 2:
            h_fwd = hn[-2]
            h_bwd = hn[-1]
            hb = torch.cat([h_fwd, h_bwd], dim=1)  # [b, 128]
        else:
            hb = hn[-1]
        # hb is [b, 128]

        # ---- Attention branch ----
        # proj -> MHA
        xa1 = self.attn_proj1(x)  # [b, seq_len, 16]
        attn_out1, _ = self.mha1(xa1, xa1, xa1)  # [b, seq_len, 16]
        a1 = attn_out1.mean(dim=1)  # pool -> [b, 16]

        xa2 = self.attn_proj2(x)
        attn_out2, _ = self.mha2(xa2, xa2, xa2)
        a2 = attn_out2.mean(dim=1)  # [b, 32]

        xa3 = self.attn_proj3(x)
        attn_out3, _ = self.mha3(xa3, xa3, xa3)
        a3 = attn_out3.mean(dim=1)  # [b, 64]

        attn_concat = torch.cat([a1, a2, a3], dim=1)  # [b, 112]

        # ---- concat and fc ----
        cat = torch.cat([xr_pool, hb, attn_concat], dim=1)
        h = self.relu(self.fc1(cat))
        h = self.dropout(h)
        h = self.relu(self.fc2(h))
        out = self.fc_out(h)
        return out.squeeze(1)

In [6]:
# -----------------------------
# Training & evaluation helpers
# -----------------------------
def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    tot_loss = 0.0
    for xb, yb in dataloader:
        xb = xb.to(device)
        yb = yb.to(device)
        optimizer.zero_grad()
        yhat = model(xb)
        loss = criterion(yhat, yb)
        loss.backward()
        optimizer.step()
        tot_loss += loss.item() * xb.size(0)
    return tot_loss / len(dataloader.dataset)

In [7]:
@torch.no_grad()
def evaluate(model, dataloader, device):
    model.eval()
    preds = []
    trues = []
    for xb, yb in dataloader:
        xb = xb.to(device)
        yb = yb.to(device)
        yhat = model(xb)
        preds.append(yhat.detach().cpu().numpy())
        trues.append(yb.detach().cpu().numpy())
    preds = np.concatenate(preds, axis=0)
    trues = np.concatenate(trues, axis=0)
    return trues, preds

In [8]:
# High-level train function
def fit(model, train_dataset, val_dataset=None, epochs=50, batch_size=64, lr=1e-3, device='cpu'):
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False) if val_dataset is not None else None
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    best_state = None
    best_val_loss = float('inf')

    for ep in range(1, epochs+1):
        loss = train_epoch(model, train_loader, criterion, optimizer, device)
        if val_loader is not None:
            y_val, y_pred = evaluate(model, val_loader, device)
            val_mse = np.mean((y_val - y_pred)**2)
            if val_mse < best_val_loss:
                best_val_loss = val_mse
                best_state = copy.deepcopy(model.state_dict())
        # optional: print progress
        if ep % 10 == 0 or ep == 1 or ep==epochs:
            print(f"[Epoch {ep}/{epochs}] train_loss={loss:.6f} val_mse={best_val_loss:.6f}")
    if best_state is not None:
        model.load_state_dict(best_state)
    return model

In [9]:
def transfer_finetune(pretrained_path: str, target_train_dataset, target_val_dataset,
                      lr=1e-4, epochs=30, batch_size=64, device='cpu', freeze_base=True):
    # Rebuild same model architecture
    sample = target_train_dataset[0][0]
    n_features = sample.shape[1]
    seq_len    = sample.shape[0]
    model = DOTransferModel(n_features=n_features, seq_len=seq_len, device=device)

    # Load pretrained weights
    state_dict = torch.load(pretrained_path, map_location=device)
    model.load_state_dict(state_dict)

    # Freeze base layers if requested
    if freeze_base:
        for name, p in model.named_parameters():
            if any(k in name for k in ['fc1', 'fc2', 'fc_out']):
                p.requires_grad = True
            else:
                p.requires_grad = False

    # Re-init FC layers
    def weights_init(m):
        if isinstance(m, nn.Linear):
            nn.init.kaiming_uniform_(m.weight, a=math.sqrt(5))
            if m.bias is not None:
                nn.init.zeros_(m.bias)
    model.fc1.apply(weights_init)
    model.fc2.apply(weights_init)
    model.fc_out.apply(weights_init)

    # Train only unfrozen parameters
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = optim.Adam(params, lr=lr)
    criterion = nn.MSELoss()
    model.to(device)

    train_loader = DataLoader(target_train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(target_val_dataset, batch_size=batch_size, shuffle=False)
    best_state, best_val = None, float('inf')

    for ep in range(1, epochs+1):
        model.train()
        tot_loss = 0.0
        for xb, yb in train_loader:
            xb, yb = xb.to(device), yb.to(device)
            optimizer.zero_grad()
            yhat = model(xb)
            loss = criterion(yhat, yb)
            loss.backward()
            optimizer.step()
            tot_loss += loss.item() * xb.size(0)

        # Validation
        y_val, y_pred = evaluate(model, val_loader, device)
        val_mse = np.mean((y_val - y_pred)**2)
        if val_mse < best_val:
            best_val, best_state = val_mse, model.state_dict().copy()

        if ep % 5 == 0:
            print(f"[FT Epoch {ep}/{epochs}] train_loss={tot_loss/len(train_loader.dataset):.6f} val_mse={val_mse:.6f}")

    if best_state is not None:
        model.load_state_dict(best_state)
    return model


In [10]:
# -----------------------------
# K-fold evaluation for small target dataset (paper used K-fold on Lake Y)
# -----------------------------
def k_fold_evaluate(model_constructor, X, y, k=5, epochs=40, device='cpu'):
    """
    model_constructor: a function that returns an uninitialized model instance.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    metrics_list = []
    for fold, (train_idx, test_idx) in enumerate(kf.split(X)):
        print(f"---- Fold {fold+1}/{k} ----")
        X_tr, X_te = X[train_idx], X[test_idx]
        y_tr, y_te = y[train_idx], y[test_idx]
        ds_tr = MyTimeSeriesDataset.from_arrays(X_tr, y_tr)
        ds_te = MyTimeSeriesDataset.from_arrays(X_te, y_te)
        model = model_constructor()
        model = fit(model, ds_tr, val_dataset=ds_te, epochs=epochs, batch_size=64, lr=1e-3, device=device)
        trues, preds = evaluate(model, DataLoader(ds_te, batch_size=64), device)
        trues = trues.flatten(); preds = preds.flatten()
        metrics = {
            'RMSE': rmse(trues, preds),
            'MAPE': mape(trues, preds),
            'R2': r2_score_np(trues, preds),
            'd': index_of_agreement(trues, preds),
            'NSE': nash_sutcliffe(trues, preds)
        }
        print(metrics)
        metrics_list.append(metrics)
    # Average metrics
    avg = {k: np.mean([m[k] for m in metrics_list]) for k in metrics_list[0]}
    print("Average metrics across folds:", avg)
    return metrics_list, avg

In [11]:
df1 = pd.read_excel('River water quality data; DO.xlsx', sheet_name='USGS 14162500')
df1 = df1.drop(columns=['Time '])

df2 = pd.read_excel('River water quality data; DO.xlsx', sheet_name='USGS 14211010')
df2 = df2.drop(columns=['Time '])

In [12]:
scaler = StandardScaler()
scaler.fit(df1.drop('DO', axis=1))

# Apply to DataFrame
scaled_array1 = scaler.transform(df1.drop("DO", axis=1))
scaled_array2 = scaler.transform(df2.drop("DO", axis=1))

In [13]:
scaled_array1.shape

(2557, 5)

In [14]:
df1['DO'].values.reshape(-1, 1)

array([[10.3],
       [10.4],
       [10.3],
       ...,
       [10.8],
       [10.8],
       [10.8]], shape=(2557, 1))

In [15]:
# Put back into DataFrame with same column names & index
df1 = pd.DataFrame(np.concatenate((scaled_array1, df1['DO'].values.reshape(-1, 1)), axis=1),
                   columns=df1.columns, index=df1.index)

# Put back into DataFrame with same column names & index
df2 = pd.DataFrame(np.concatenate((scaled_array2, df2['DO'].values.reshape(-1, 1)), axis=1),
                   columns=df2.columns, index=df2.index)


In [16]:
# -----------------------------
# 1. Sequence builder for both lakes
# -----------------------------
def make_sequences(df, seq_len=60, pred_horizon=60):
    features = df[['WT','Q','SC','PH','Tu']].values
    target   = df['DO'].values
    X, y = [], []
    for i in range(len(df) - seq_len - pred_horizon):
        X.append(features[i:i+seq_len])
        y.append(target[i+seq_len+pred_horizon-1])
    return np.array(X, dtype=np.float32), np.array(y, dtype=np.float32)

In [17]:
df1.shape

(2557, 6)

In [18]:
df2.shape

(2557, 6)

In [19]:
df1_train, df1_test = df1[:2200], df1[2200:]
df2_train, df2_test = df2[:2200], df2[2200:]

In [20]:
seq_len, pred_horizon = 60, 60

X1, y1 = make_sequences(df1_train, seq_len, pred_horizon)   # Lake T
X2, y2 = make_sequences(df2_train, seq_len, pred_horizon)   # Lake Y

In [21]:
X1_test, y1_test = make_sequences(df1_test, seq_len, pred_horizon) 
X2_test, y2_test = make_sequences(df2_test, seq_len, pred_horizon) 

In [22]:
# -----------------------------
# 2. Train on Lake T (pretraining)
# -----------------------------
train_ds1 = MyTimeSeriesDataset.from_arrays(X1, y1)

# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'
def make_model(): return DOTransferModel(n_features=X1.shape[2], seq_len=seq_len, device=device)

print("Pretraining on Lake T...")
pre_model = make_model()
pre_model = fit(pre_model, train_ds1, val_dataset=None,
                epochs=50, batch_size=128, lr=1e-3, device=device)

torch.save(pre_model.state_dict(), "lakeT_pretrained.pth")

Pretraining on Lake T...
[Epoch 1/50] train_loss=72.621436 val_mse=inf
[Epoch 10/50] train_loss=0.676872 val_mse=inf
[Epoch 20/50] train_loss=0.464859 val_mse=inf
[Epoch 30/50] train_loss=0.413393 val_mse=inf
[Epoch 40/50] train_loss=0.387647 val_mse=inf
[Epoch 50/50] train_loss=0.395033 val_mse=inf


In [23]:
# -----------------------------
# 3. Lake Y splits
# -----------------------------
X2_train, X2_val, y2_train, y2_val = train_test_split(X2, y2, test_size=0.2, shuffle=False)
train_ds2 = MyTimeSeriesDataset.from_arrays(X2_train, y2_train)
val_ds2   = MyTimeSeriesDataset.from_arrays(X2_val,   y2_val)

# -----------------------------
# 4A. Transfer learning on Lake Y
# -----------------------------
print("\nFine-tuning (transfer learning) on Lake Y...")
ft_model = transfer_finetune("lakeT_pretrained.pth",
                             target_train_dataset=train_ds2,
                             target_val_dataset=val_ds2,
                             lr=5e-4, epochs=60, batch_size=32,
                             device=device, freeze_base=True)

y_true_ft, y_pred_ft = evaluate(ft_model, DataLoader(val_ds2, batch_size=64), device)
metrics_ft = {
    "RMSE": rmse(y_true_ft, y_pred_ft),
    "MAPE": mape(y_true_ft, y_pred_ft),
    "R2":   r2_score_np(y_true_ft, y_pred_ft),
    "d":    index_of_agreement(y_true_ft, y_pred_ft),
    "NSE":  nash_sutcliffe(y_true_ft, y_pred_ft)
}


Fine-tuning (transfer learning) on Lake Y...
[FT Epoch 5/60] train_loss=0.987344 val_mse=0.438083
[FT Epoch 10/60] train_loss=0.984240 val_mse=0.345338
[FT Epoch 15/60] train_loss=1.012057 val_mse=0.395659
[FT Epoch 20/60] train_loss=0.895623 val_mse=0.448899
[FT Epoch 25/60] train_loss=0.871433 val_mse=0.374025
[FT Epoch 30/60] train_loss=0.842582 val_mse=0.518583
[FT Epoch 35/60] train_loss=0.808310 val_mse=0.451760
[FT Epoch 40/60] train_loss=0.695780 val_mse=0.585027
[FT Epoch 45/60] train_loss=0.647125 val_mse=0.536099
[FT Epoch 50/60] train_loss=0.600480 val_mse=1.372382
[FT Epoch 55/60] train_loss=0.490912 val_mse=1.548131
[FT Epoch 60/60] train_loss=0.488483 val_mse=2.459424


In [24]:
# -----------------------------
# 4B. Training from scratch on Lake Y
# -----------------------------
print("\nTraining from scratch on Lake Y...")
scratch_model = make_model()
scratch_model = fit(scratch_model, train_ds2, val_dataset=val_ds2,
                    epochs=50, batch_size=32, lr=1e-3, device=device)

y_true_sc, y_pred_sc = evaluate(scratch_model, DataLoader(val_ds2, batch_size=64), device)
metrics_sc = {
    "RMSE": rmse(y_true_sc, y_pred_sc),
    "MAPE": mape(y_true_sc, y_pred_sc),
    "R2":   r2_score_np(y_true_sc, y_pred_sc),
    "d":    index_of_agreement(y_true_sc, y_pred_sc),
    "NSE":  nash_sutcliffe(y_true_sc, y_pred_sc)
}


Training from scratch on Lake Y...
[Epoch 1/50] train_loss=23.508194 val_mse=1.382350
[Epoch 10/50] train_loss=0.575694 val_mse=0.177256
[Epoch 20/50] train_loss=0.461997 val_mse=0.177256
[Epoch 30/50] train_loss=0.361116 val_mse=0.177256
[Epoch 40/50] train_loss=0.235467 val_mse=0.177256
[Epoch 50/50] train_loss=0.126602 val_mse=0.177256


In [25]:
# -----------------------------
# 5. Compare results
# -----------------------------
print("\n=== Results on Lake Y ===")
print("Transfer learning:", metrics_ft)
print("From scratch:     ", metrics_sc)



=== Results on Lake Y ===
Transfer learning: {'RMSE': 1.5682551860809326, 'MAPE': 12.741239547729492, 'R2': -0.18608629703521729, 'd': 0.7412799596786499, 'NSE': -0.18608629703521729}
From scratch:      {'RMSE': 0.4210173487663269, 'MAPE': 2.9985365867614746, 'R2': 0.9145163893699646, 'd': 0.9761101007461548, 'NSE': 0.9145163893699646}
